In [6]:
from environments.h2000_v90 import citation
import torch
import numpy as np

In [10]:
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [6]:
print(citation.initialize())

None


In [2]:
torch.__version__

'2.0.0+cu117'

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.version.cuda

'11.7'

In [11]:
citation.initialize()

input = np.zeros((10,))
input

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [13]:
x = citation.step(input)

In [14]:
x

array([ 5.14511340e-15, -3.10150901e-03, -2.66269961e-15,  8.99998899e+01,
        5.59248529e-02, -4.84839698e-14,  2.59608529e-17,  3.66017228e-03,
       -1.33996579e-17,  1.99995297e+03,  8.98769684e-01, -4.36894623e-14])

In [16]:
error = np.zeros(3)
error

array([0., 0., 0.])

In [20]:
obs = np.hstack((error, x[range(10)]))

In [22]:
obs.shape, obs

((13,),
 array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.14511340e-15,
        -3.10150901e-03, -2.66269961e-15,  8.99998899e+01,  5.59248529e-02,
        -4.84839698e-14,  2.59608529e-17,  3.66017228e-03, -1.33996579e-17,
         1.99995297e+03]))

In [23]:
np.deg2rad(45)

0.7853981633974483

In [63]:
import jsbsim
import time

In [105]:
fdm = jsbsim.FGFDMExec("environments/JSBSim", None)
fdm.load_model("citation")
fdm.set_output_directive("data_output/flightgear.xml")
fdm.load_ic("cruise_init", True)
fdm.run_ic()
fdm["gear/gear-cmd-norm"] = 0
for _ in range(10):
    fdm.run()
fdm.do_trim(1)

# while fdm.run() and fdm.get_sim_time() < 100:
#     print(fdm.get_sim_time())
#     fdm.run()
#     time.sleep(fdm.get_delta_t())
   




     JSBSim Flight Dynamics Model v1.1.13 Dec  3 2022 12:34:26
            [JSBSim-ML v2.0]

JSBSim startup beginning ...

Reading Aircraft Configuration File: Cessna Citation II 550 Jet Passenger Aircraft
                            Version: 2.0


This aircraft model is a BETA release!!!

This aircraft model probably will not fly as expected.

Use this model for development purposes ONLY!!!

  Description:   Models a Cessna Citation II 550.
  Model Author:  Aeromatic v 3.3.11
  Creation Date: 2020-04-09
  Version:       $Revision: 1.81 $

  Aircraft Metrics:
    WingArea: 323.000000
    WingSpan: 51.700000
    Incidence: 0.017453
    Chord: 6.680000
    H. Tail Area: 140.400000
    H. Tail Arm: 42.400000
    V. Tail Area: 47.100000
    V. Tail Arm: 43.600000
    Eyepoint (x, y, z): 39.690000 , -30.000000 , 70.000000
    Ref Pt (x, y, z): 276.132000 , 0.000000 , 0.000000
    Visual Ref Pt (x, y, z): 0.000000 , 0.000000 , 0.000000

  Mass and Balance:
    baseIxx: 9140.000000 slug-ft2


In file environments/JSBSim/aircraft/citation/citation.xml: line 232
Engine location ignored, only thruster location is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 237
Engine orientation ignored, only thruster orientation is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 260
Engine location ignored, only thruster location is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 265
Engine orientation ignored, only thruster orientation is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1311
      Property fcs/aileron-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1312
      Property fcs/elevator-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1313
      Property fcs/rudder-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1314
      Property fcs/throttle-

In [96]:
fdm.get_property_catalog()

['inertial/sea-level-radius_ft (R)',
 'simulation/gravity-model (RW)',
 'simulation/integrator/rate/rotational (RW)',
 'simulation/integrator/rate/translational (RW)',
 'simulation/integrator/position/rotational (RW)',
 'simulation/integrator/position/translational (RW)',
 'simulation/write-state-file (W)',
 'simulation/channel-dt (R)',
 'simulation/gravitational-torque (RW)',
 'simulation/force-output (W)',
 'simulation/do_simple_trim (W)',
 'simulation/reset (W)',
 'simulation/disperse (R)',
 'simulation/randomseed (RW)',
 'simulation/terminate (RW)',
 'simulation/pause (RW)',
 'simulation/sim-time-sec (R)',
 'simulation/dt (R)',
 'simulation/jsbsim-debug (RW)',
 'simulation/frame (RW)',
 'simulation/trim-completed (RW)',
 'simulation/output/log_rate_hz (RW)',
 'simulation/output/enabled (RW)',
 'velocities/h-dot-fps (R)',
 'velocities/v-north-fps (R)',
 'velocities/v-east-fps (R)',
 'velocities/v-down-fps (R)',
 'velocities/u-fps (R)',
 'velocities/v-fps (R)',
 'velocities/w-fps (R)

In [97]:
print({
    "elevator": [fdm['fcs/elevator-pos-rad'], fdm['fcs/elevator-cmd-norm']],
    "left-aileron": [fdm['fcs/left-aileron-pos-rad'], fdm['fcs/aileron-cmd-norm']],
    "rudder": [fdm['fcs/rudder-pos-rad'], fdm['fcs/rudder-cmd-norm']]
})


{'elevator': [-0.03227545188287851, 0.0], 'left-aileron': [-5.688945793439429e-07, -1.7135378895901894e-06], 'rudder': [-4.552028747622394e-06, -1.1854241530266652e-05]}


In [98]:
action = np.array([0.5, 0.25, 0.5])
[de, da, dr] = action
de, da, dr

(0.5, 0.25, 0.5)

In [99]:
[de_cmd, da_cmd, dr_cmd] = np.array([0.5, 0.6, 0.25])

In [100]:

pause_delay = 0.1
physics_per_sec = int(1/fdm.get_delta_t())
realtime_delay = fdm.get_delta_t() 
realtime = True
# fdm['fcs/left-aileron-pos-rad'] = da

# fdm['fcs/right-aileron-pos-rad'] = -da

# fdm['fcs/elevator-pos-rad'] = de

# fdm['fcs/rudder-pos-rad'] = dr
for i in range(100):
    fdm['fcs/aileron-cmd-norm'] = da_cmd
    fdm['fcs/elevator-cmd-norm'] = de_cmd
    fdm['fcs/rudder-cmd-norm'] = dr_cmd
    for i in range(int(pause_delay*physics_per_sec)):
        fdm['fcs/aileron-cmd-norm'] = da_cmd
        fdm['fcs/elevator-cmd-norm'] = de_cmd
        fdm['fcs/rudder-cmd-norm'] = dr_cmd
        fdm.run()
        if realtime:
            time.sleep(realtime_delay)
    
    print({
        "elevator": [fdm['fcs/elevator-pos-rad'], fdm['fcs/elevator-cmd-norm']],
        "left-aileron": [fdm['fcs/left-aileron-pos-rad'], fdm['fcs/aileron-cmd-norm']],
        "rudder": [fdm['fcs/rudder-pos-rad'], fdm['fcs/rudder-cmd-norm']]
    })



{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008073, 0.5], 'left-aileron': [0.1782, 0.6], 'rudder': [0.096, 0.25]}
{'elevator': [0.11499520734008

In [2]:

from environments.aircraftenv import AircraftEnv


In [5]:
env = AircraftEnv(configuration="full_control", render_mode=True, realtime=True, incremental=False)

Full State Control.



     JSBSim Flight Dynamics Model v1.1.13 Dec  3 2022 12:34:26
            [JSBSim-ML v2.0]

JSBSim startup beginning ...

Reading Aircraft Configuration File: Cessna Citation II 550 Jet Passenger Aircraft
                            Version: 2.0


This aircraft model is a BETA release!!!

This aircraft model probably will not fly as expected.

Use this model for development purposes ONLY!!!

  Description:   Models a Cessna Citation II 550.
  Model Author:  Aeromatic v 3.3.11
  Creation Date: 2020-04-09
  Version:       $Revision: 1.81 $

  Aircraft Metrics:
    WingArea: 323.000000
    WingSpan: 51.700000
    Incidence: 0.017453
    Chord: 6.680000
    H. Tail Area: 140.400000
    H. Tail Arm: 42.400000
    V. Tail Area: 47.100000
    V. Tail Arm: 43.600000
    Eyepoint (x, y, z): 39.690000 , -30.000000 , 70.000000
    Ref Pt (x, y, z): 276.132000 , 0.000000 , 0.000000
    Visual Ref Pt (x, y, z): 0.000000 , 0.000000 , 0.000000

  Mass and Balance:
    baseIxx:


In file environments/JSBSim/aircraft/citation/citation.xml: line 232
Engine location ignored, only thruster location is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 237
Engine orientation ignored, only thruster orientation is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 260
Engine location ignored, only thruster location is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 265
Engine orientation ignored, only thruster orientation is used.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1311
      Property fcs/aileron-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1312
      Property fcs/elevator-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1313
      Property fcs/rudder-cmd-norm is already defined.

In file environments/JSBSim/aircraft/citation/citation.xml: line 1314
      Property fcs/throttle-

KeyboardInterrupt: 

target_wp_longitude_rad (initial value: 0.0000)

      navigation/actual-heading-rad (initial value: 0.0000)

      guidance/specified-heading-rad (initial value: 0.0000)

      guidance/heading-selector-switch (initial value: 0.0000)


  System: GNC Utilities

    Channel Guidance Waypoint Heading Director

    Loading Component "guidance/wp-heading-rad" of type: WAYPOINT_HEADING

    Loading Component "guidance/wp-distance" of type: WAYPOINT_DISTANCE

    Loading Component "guidance/wp-heading-deg" of type: PURE_GAIN
      INPUT: wp-heading-rad
      GAIN: constant value 57.300000
      OUTPUT: wp-heading-deg

    Channel Included Angle to Heading

    Loading Component "guidance/selected_target_heading" of type: SWITCH
      Switch default value is: wp-heading-rad
      Switch takes test 1 value (specified-heading-rad)
      if all of the following are true: {
        heading-selector-switch eq constant value 1.000000
      }
      OUTPUT: selected_target_heading

    Loading Compon

In [ ]:
env.action_space

Box(-0.17453292519943295, 0.17453292519943295, (3,), float64)

In [ ]:
env.observation_space

Box(-30.0, 30.0, (13,), float64)

In [ ]:
env.reset()

Creating input TCP socket on port 5137
Creating input UDP socket on port 5139
Creating UDP socket on port 5550
Successfully connected to socket for output ...

  Mass Properties Report (English units: lbf, in, slug-ft^2)
                                      Weight    CG-X    CG-Y    CG-Z         Ixx         Iyy         Izz         Ixy         Ixz         Iyz
    Base Vehicle                      7388.0   273.0     0.0    -1.9      9140.0     23234.0     30910.0        -0.0         0.0        -0.0
0   Front Baggage                        0.0    63.0     0.0    15.7         0.0         0.0         0.0        -0.0         0.0        -0.0
1   Pilot                              176.4   122.0   -13.8    21.7         0.0         0.0         0.0        -0.0         0.0        -0.0
2   Copilot                            176.4   122.0    13.8    21.7         0.0         0.0         0.0        -0.0         0.0        -0.0
3   Passengers                         352.7  

Could not bind to TCP input socket, error = 98
Could not bind to UDP input socket, error = 98


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  9.00000000e+01,  5.59757360e-02,
       -4.85893135e-14,  0.00000000e+00,  3.67573600e-03,  0.00000000e+00,
        2.00000000e+03])

 236.2     0.0    19.7         0.0         0.0         0.0        -0.0         0.0        -0.0
4   Rear Baggage 1                       0.0   302.9     0.0    12.0         0.0         0.0         0.0        -0.0         0.0        -0.0
5   Rear Baggage 2                       0.0   320.2     0.0    12.0         0.0         0.0         0.0        -0.0         0.0        -0.0
6   tailcone                             0.0   423.6     0.0    24.0         0.0         0.0         0.0        -0.0         0.0        -0.0
0   Fuel                             669.398     280     -96    -0.4           0           0           0
1   Fuel                             669.398     280      96    -0.4           0           0           0
2   Fuel                                   3     280     -96    -0.4           0           0           0
3   Fuel                                   3     280      96    -0.4           0           0           0
                                                              

In [ ]:
import numpy as np
env.step(np.array([0.1, 0.1, 0.05]))

NameError: name 'env' is not defined